In [15]:
import os
import sys
import numpy as np
import wandb

from playlist_recommender.modelling import model_pipeline
from playlist_recommender.modelling import utils
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from wandb.keras import WandbCallback
import tensorflow as tf
from tensorflow import keras
from keras import layers
from pprint import pprint

In [16]:
X, y = utils.prep_playlist_df()
X_train, X_test, y_train, y_test = model_pipeline.make_best_transformation_pipeline(
    X, y
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape,

((7293, 1047), (3927, 1047), (7293,), (3927,))

In [17]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

# WandB Sweeps

In [18]:
sweep_config = {
    'method': 'grid'
    }

In [19]:
metric = {
    'name': 'f1_score',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

In [20]:
parameters_dict = {
    'fc_layer_size': {
        'values': [64,128,256,]
        },
    }

sweep_config['parameters'] = parameters_dict

In [21]:
parameters_dict.update({
    'epochs': {
        'values': [150,300,500]},
    'learning_rate': {
      'values':[0.00001, 0.001, 0.01]
      },
    'batch_size': {'values': [32,64]}
    })

In [22]:
pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'f1_score'},
 'parameters': {'batch_size': {'values': [32, 64]},
                'epochs': {'values': [150, 300, 500]},
                'fc_layer_size': {'values': [64, 128, 256]},
                'learning_rate': {'values': [1e-05, 0.001, 0.01]}}}


In [23]:
sweep_id = wandb.sweep(sweep_config, project="spotify-recommender")

Create sweep with ID: su14o02h
Sweep URL: https://wandb.ai/jmoro/spotify-recommender/sweeps/su14o02h


In [24]:
def build_model(fc_layer_size=15):
    input_shape = X_train.shape[1]
    num_classes = len(le.classes_)
    return keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(fc_layer_size, activation="relu"),
        layers.Dense(fc_layer_size, activation="relu"),
        layers.Dense(fc_layer_size, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)


In [25]:
def train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'epochs':30,
        'batch_size': 8,
        'learning_rate': 0.01,
        'fc_layer_size':15
    }

    # Initialize wandb with a sample project name
    with wandb.init(
        project="spotify-recommender",
        tags=["nn"],
        name="neural network tuning",
        config=wandb.config,
    ):
        config = wandb.config
        # Iniialize model with hyperparameters
        keras.backend.clear_session()
        model = build_model(
                        fc_layer_size=wandb.config.fc_layer_size)

        # Compile the model
        opt = tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate) # optimizer with different learning rate specified by config
        model.compile(opt, 'sparse_categorical_crossentropy', metrics=['acc'])

        # Train the model
        model.fit(X_train, y_train,
                        epochs=wandb.config.epochs, 
                        validation_data=(X_test, y_test),
                        callbacks=[WandbCallback(
                            save_model = False, 
                            save_graph = False, 
                            log_weights = False, 
                            log_gradients = False, 
                            )]) # WandbCallback to automatically track metrics

        y_pred = np.argmax(model.predict(X_test), axis=-1)
        f1_score = metrics.f1_score(y_test, y_pred, average="macro", zero_division=0)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(
            y_test, y_pred, average="macro", zero_division=0
        )
        recall = metrics.recall_score(y_test, y_pred, average="macro")
        wandb.log(
            {
                "f1_score": f1_score,
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
            }
        )
                            

In [26]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 4h2zc7e0 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 150
wandb: 	fc_layer_size: 64
wandb: 	learning_rate: 1e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


2022-09-26 08:26:33.288875: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-09-26 08:26:33.289101: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


Epoch 1/150


/opt/miniconda3/envs/playlist-creator/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


228/228 [==============================] - 9s 15ms/step - loss: 3.4961 - acc: 0.0317 - val_loss: 3.4950 - val_acc: 0.0364
Epoch 2/150
228/228 [==============================] - 3s 11ms/step - loss: 3.4937 - acc: 0.0358 - val_loss: 3.4927 - val_acc: 0.0413
Epoch 3/150
228/228 [==============================] - 2s 10ms/step - loss: 3.4914 - acc: 0.0426 - val_loss: 3.4904 - val_acc: 0.0466
Epoch 4/150
228/228 [==============================] - 2s 10ms/step - loss: 3.4889 - acc: 0.0446 - val_loss: 3.4879 - val_acc: 0.0514
Epoch 5/150
228/228 [==============================] - 2s 10ms/step - loss: 3.4862 - acc: 0.0503 - val_loss: 3.4853 - val_acc: 0.0568
Epoch 6/150
228/228 [==============================] - 3s 11ms/step - loss: 3.4834 - acc: 0.0607 - val_loss: 3.4825 - val_acc: 0.0693
Epoch 7/150
228/228 [==============================] - 2s 9ms/step - loss: 3.4803 - acc: 0.0728 - val_loss: 3.4793 - val_acc: 0.0782
Epoch 8/150
228/228 [==============================] - 2s 10ms/step - loss:

...

Epoch 1/500


2022-09-26 17:12:58.559018: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-09-26 17:12:58.559260: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
/opt/miniconda3/envs/playlist-creator/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 256), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


228/228 [==============================] - 6s 14ms/step - loss: 1.6692 - acc: 0.4990 - val_loss: 0.8130 - val_acc: 0.7622
Epoch 2/500
228/228 [==============================] - 2s 8ms/step - loss: 0.6782 - acc: 0.7965 - val_loss: 0.7762 - val_acc: 0.7751
Epoch 3/500
228/228 [==============================] - 2s 7ms/step - loss: 0.5500 - acc: 0.8361 - val_loss: 0.7127 - val_acc: 0.7968
Epoch 4/500
228/228 [==============================] - 2s 7ms/step - loss: 0.4502 - acc: 0.8588 - val_loss: 0.6709 - val_acc: 0.8245
Epoch 5/500
228/228 [==============================] - 2s 7ms/step - loss: 0.4217 - acc: 0.8625 - val_loss: 0.5656 - val_acc: 0.8296
Epoch 6/500
228/228 [==============================] - 2s 7ms/step - loss: 0.3692 - acc: 0.8791 - val_loss: 0.6095 - val_acc: 0.8411
Epoch 7/500
228/228 [==============================] - 2s 8ms/step - loss: 0.3740 - acc: 0.8774 - val_loss: 0.5861 - val_acc: 0.8416
Epoch 8/500
228/228 [==============================] - 2s 7ms/step - loss: 0.339

acc,▁▅▅▆▇▆███▇▇▇██▇██▇▄████▇▇▇▇█▇▄▄▆▇▆▅▇▇▇▇▇
accuracy,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁
loss,▇▃▄▃▂▃▁▁▁▂▃▃▁▂▃▁▁▂█▁▂▂▁▁▂▂▂▂▃▆▄▂▂▂▆▂▂▃▃▂
precision,▁
recall,▁
val_acc,▁▆▃▂▆▄▇▇█▆▂▄▆▇▇█▆▆▃▅▆▆▇▇▄▅▆▇▄▅▃▆▆▅▅▇▆▄▃▆
val_loss,▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅█▆▅▅▇
GFLOPs,0.00041
acc,0.91458


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
